In [114]:
!pip install sbnltk
!pip install simpletransformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 47.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 65.4 MB/s eta 0:00:00:00:01
  Created wheel for sbnltk: filename=sbnltk-1.0.7-py3-none-any.whl size=31681 sha256=71e2cad8e8c8048e82b729f14a8dd04ec265fffeb3b67d65d35e29e598cf39f0
  Stored in directory: /root/.cache/pip/wheels/15/4d/af/47d3e47dd41321f36547e58ecaac6f89c157602415d1775e34
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4baa10a1c810aadb430467ea22b79fe1e52d320f55f058a6107b94a1098278b8
  Stored in directory: /root/.cache/pip/wheels/bf/06/f

In [115]:
!pip uninstall scikit-learn -y

Found existing installation: scikit-learn 0.22.2.post1
Uninstalling scikit-learn-0.22.2.post1:
  Successfully uninstalled scikit-learn-0.22.2.post1


In [116]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 33.6 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sbnltk 1.0.7 requires scikit-learn==0.22.2.post1, but you have scikit-learn 1.0.2 which is incompatible.
imbalanced-learn 0.10.1 requires joblib>=1.1.1, but you have joblib 1.0.1 which is incompatible.
featuretools 1.11.1 requires numpy>=1.21.0, but you have numpy 1.19.5 which is incompatible.
allennlp 2.10.1 requires h5py>=3.6.0, but you have h5py 3.1.0 which is incompatible.
allennlp 2.10.1 requires numpy>=1.21.4, but you have numpy 1.19.5 which is incompatible.


In [117]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

In [141]:
import numpy as np
import string 

punctuations = list(string.punctuation) + ['।']
print(punctuations)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '।']


In [193]:
def load_data(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.strip().split(' _ _ ') for line in f.readlines()]
        # return data
        # empty lines are used to separate sentences
        # separate them into sentences
        sentences = []
        cur = []
        for line in data:
            if line == ['']:
                cur = [tuple(line) for line in cur]
                sentences.append(cur)
                cur = []
            else:
#                 for p in punctuations:
#                     line[0] = line[0].replace(p, '')
                if len(line[0]) == 0:
                    line[0] = ' '
                cur.append(line)
        # # convert each list to a tuple
        sentences.append(cur)
        return sentences

In [194]:
def load_words(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.strip().split(' _ _ ') for line in f.readlines()]
        words = []
        for line in data:
            if line == ['']:
                continue
            else:
                words.append(line[0])
        # # convert each list to a tuple
        return list(set(words))

In [195]:
train_words_set = load_words('../data/train.txt')
dev_words_set = load_words('../data/dev.txt')

In [196]:
len(dev_words_set), dev_words_set[:10]

(4713,
 ['হারমোনিকা',
  'চেয়ারটি',
  'উৎপাদিত',
  'সম্পর্কে।',
  'ভোকাল',
  'বস্তুবাদে',
  'পরেন।',
  'রাজকীয়ভাবে',
  'সুখী',
  'রিজেনরন'])

In [197]:
import os
os.listdir('../data/')

['location.txt',
 'album_names_bn.txt',
 'location_names_bn.txt',
 'test.txt',
 'names_bn.txt',
 'gazetters',
 'train.txt',
 'dev.txt',
 'test_labels_dummy.txt',
 'location_small.txt',
 'movies_bn.txt']

In [198]:
from sbnltk.word_embedding import gensim_word2vec_embedding
w2v=gensim_word2vec_embedding()

In [199]:
def get_w2v(words):
    ret = []
    for word in words:
        ret.append(w2v.get_vector(word))
    return np.asarray(ret)

def get_cluster_id(words):
    words_w2v = get_w2v(words)
    kmeans = KMeans(n_clusters=500, random_state=0).fit(words_w2v)
    return kmeans.labels_, kmeans

In [200]:
train_words_w2v = get_w2v(train_words_set)

In [201]:
dev_words_w2v = get_w2v(dev_words_set)

In [202]:
# gmm = GaussianMixture(n_components=500, random_state=0).fit(train_words_w2v)

In [203]:
kmeans = KMeans(n_clusters=500, random_state=0).fit(train_words_w2v)

In [204]:
kmeans.labels_

array([489, 489,   4, ...,   7,   4, 485], dtype=int32)

In [205]:
kmeans.predict(train_words_w2v[0:2])

array([489, 489], dtype=int32)

In [206]:
# gmm.predict(train_words_w2v[0:2])

In [207]:
train = load_data('../data/train.txt')

In [208]:
dev = load_data('../data/dev.txt')

In [209]:
dev[6]

[('আরিয়ানা', 'B-PER'),
 ('গ্রান্দে', 'I-PER'),
 ('-', 'O'),
 ('লিড', 'O'),
 ('ভোকাল,', 'O'),
 ('ব্যাকগ্রাউন্ড', 'O'),
 ('ভোকাল,', 'O'),
 ('গান', 'O'),
 ('রচনা,', 'O'),
 ('ভোকাল', 'O'),
 ('প্রোডাকশন,', 'O'),
 ('ভোকাল', 'O'),
 ('অ্যারেঞ্জমেন্ট,', 'O'),
 ('অডিও', 'O'),
 ('ইঞ্জিনিয়ারিং', 'O')]

In [210]:
train[0]

[('তার', 'O'),
 ('মৃত্যুর', 'O'),
 ('দশ', 'O'),
 ('দিন', 'O'),
 ('পর,', 'O'),
 ('১১৫', 'O'),
 ('কৃষ্ণাঙ্গ', 'O'),
 ('উচ্চ', 'O'),
 ('বিদ্যালয়ের', 'O'),
 ('শিক্ষার্থীরা', 'O'),
 ('তার', 'O'),
 ('হত্যার', 'O'),
 ('প্রতিবাদে', 'O'),
 ('ম্যাককম্ব', 'B-LOC'),
 ('এর', 'O'),
 ('মাধ্যমে', 'O'),
 ('মিছিল', 'O'),
 ('করেছে।', 'O')]

In [211]:
!pip install bnlp_toolkit

In [212]:
!git clone https://github.com/sagorbrur/bnlp

fatal: destination path 'bnlp' already exists and is not an empty directory.


In [213]:
from bnlp import POS
bn_pos = POS()
model_path = "/kaggle/working/bnlp/model/bn_pos.pkl"
text = "আমি ভাত খাই।" # or you can pass ['আমি', 'ভাত', 'খাই', '।']
res = bn_pos.tag(model_path, text)
print(res)
# [('আমি', 'PPR'), ('ভাত', 'NC'), ('খাই', 'VM'), ('।', 'PU')]

[('আমি', 'PPR'), ('ভাত', 'NC'), ('খাই', 'VM'), ('।', 'PU')]


In [214]:
def add_pos(sentence):
    # first one of the tuple is the word and the second one is the ner  
    words = []
    for word in sentence:
        words.append(word[0])
    pos = bn_pos.tag(model_path, words)
    ## add ner back to the tuple
    ret = []
    for i in range(len(pos)):
        word = sentence[i][0]
        ner = sentence[i][1]
        ret.append((word, pos[i][1], ner))
    return ret

def add_pos_to_all(sents):
    ret = []
    for i in range(len(sents)):
        if i % 100 == 0:
            print(i)
        ret.append(add_pos(sents[i]))
    return ret

In [215]:
add_pos(train[0])

[('তার', 'PPR', 'O'),
 ('মৃত্যুর', 'NC', 'O'),
 ('দশ', 'JQ', 'O'),
 ('দিন', 'NC', 'O'),
 ('পর,', 'NC', 'O'),
 ('১১৫', 'RDF', 'O'),
 ('কৃষ্ণাঙ্গ', 'NC', 'O'),
 ('উচ্চ', 'JJ', 'O'),
 ('বিদ্যালয়ের', 'NC', 'O'),
 ('শিক্ষার্থীরা', 'NC', 'O'),
 ('তার', 'PPR', 'O'),
 ('হত্যার', 'NC', 'O'),
 ('প্রতিবাদে', 'NC', 'O'),
 ('ম্যাককম্ব', 'NC', 'B-LOC'),
 ('এর', 'PPR', 'O'),
 ('মাধ্যমে', 'PP', 'O'),
 ('মিছিল', 'NC', 'O'),
 ('করেছে।', 'VM', 'O')]

In [216]:
train_sents =  add_pos_to_all(train)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200


In [217]:
dev_sents = add_pos_to_all(dev)

0
100
200
300
400
500
600
700


In [218]:
train_sents[0]

[('তার', 'PPR', 'O'),
 ('মৃত্যুর', 'NC', 'O'),
 ('দশ', 'JQ', 'O'),
 ('দিন', 'NC', 'O'),
 ('পর,', 'NC', 'O'),
 ('১১৫', 'RDF', 'O'),
 ('কৃষ্ণাঙ্গ', 'NC', 'O'),
 ('উচ্চ', 'JJ', 'O'),
 ('বিদ্যালয়ের', 'NC', 'O'),
 ('শিক্ষার্থীরা', 'NC', 'O'),
 ('তার', 'PPR', 'O'),
 ('হত্যার', 'NC', 'O'),
 ('প্রতিবাদে', 'NC', 'O'),
 ('ম্যাককম্ব', 'NC', 'B-LOC'),
 ('এর', 'PPR', 'O'),
 ('মাধ্যমে', 'PP', 'O'),
 ('মিছিল', 'NC', 'O'),
 ('করেছে।', 'VM', 'O')]

In [219]:
dev_sents[0]

[('তিনি', 'PPR', 'O'),
 ('যুবক', 'NC', 'O'),
 ('হিসেবে', 'PP', 'O'),
 ('শেফিল্ড', 'NP', 'B-GRP'),
 ('বুধবার', 'NP', 'I-GRP'),
 ('এফ.সি.', 'NP', 'I-GRP'),
 ('যোগদান', 'NC', 'O'),
 ('করেন', 'VM', 'O'),
 ('এবং', 'CCD', 'O'),
 ('১৯৫১', 'RDF', 'O'),
 ('সালে', 'NC', 'O'),
 ('তাদের', 'PPR', 'O'),
 ('পেশাগত', 'JJ', 'O'),
 ('দিক', 'NC', 'O'),
 ('দিয়ে', 'NC', 'O'),
 ('আত্মপ্রকাশ', 'NC', 'O'),
 ('করেন।', 'VM', 'O')]

In [220]:
!pip3 install tensorflow-hub

In [221]:
def get_all_words(sents):
    words = []
    for i in range(len(sents)):
        for w in sents[i]:
            words += [w[0]]
    return words

In [222]:
all_train_words = get_all_words(train_sents)
len(all_train_words)

191897

In [223]:
word_freq = {}
for w in all_train_words:
    if w in word_freq:
        word_freq[w] += 1
    else:
        word_freq[w] = 1

In [224]:
# Load the Gazetters

album_names_bn = {}
movies_bn = {}
names_bn = {}
locations_bn = {}

with open('../data/album_names_bn.txt') as file:
    lines = [x.strip() for x in file.readlines()]
    for l in lines:
        album_names_bn[l] = 1
        words = l.split()
        for size in range(2, 4, 1):
            for i in range(len(words)):
                if (i + size) >= len(words):
                    break
                album_names_bn[" ".join(words[i: i+size])] = 1

with open('../data/movies_bn.txt') as file:
    lines = [x.strip() for x in file.readlines()]
    for l in lines:
        movies_bn[l] = 1
        words = l.split()
        for size in range(2, 4, 1):
            for i in range(len(words)):
                if (i + size) > len(words):
                    break
                movies_bn[" ".join(words[i: i+size])] = 1

with open('../data/location_names_bn.txt') as file:
    lines = [x.strip() for x in file.readlines()]
    for l in lines:
        locations_bn[l] = 1
        words = l.split()
        for size in range(2, 4, 1):
            for i in range(len(words)):
                if (i + size) >= len(words):
                    break
                locations_bn[" ".join(words[i: i+size])] = 1


with open('../data/names_bn.txt') as file:
    lines = [x.strip() for x in file.readlines()]
    for l in lines:
        names_bn[l] = 1 

In [225]:
## feature extraction for Conditional Random Field - Bangla NER
## feature extraction for Conditional Random Field - Bangla NER

def wordToFeatures(sent, idx):
    word = sent[idx][0]
    postag = sent[idx][1]
    
    cluster_id = kmeans.predict([w2v.get_vector(word)])[0]
    features = {
        'bias': 1.0,
        'word': word,
        'cluster_id': cluster_id,
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word.isdigit': word.isdigit(),
        'index': idx,
        'length': len(word),
        'postag': postag,
        'freq': word_freq[word] if word in word_freq else 0,
#         'is_name': int(word in names_bn), 
#         'movie_name': int(word in movies_bn),
#         'album_name': int(word in album_names_bn),
    }
    
#     if idx > 0:
#         sub = sent[idx-1][0] + " "+ sent[idx][0]
#         if sub in movies_bn:
#             features['movie_name'] = 2
#         if sub in album_names_bn:
#             features['album_name'] = 2
    
#     if idx < len(sent) - 1:
#         sub = sent[idx][0] + " "+ sent[idx+1][0]
#         if sub in movies_bn:
#             features['movie_name'] = 2
#         if sub in album_names_bn:
#             features['album_name'] = 2
    
#     if idx > 1:
#         sub = sent[idx-2][0] + " "+ sent[idx-1][0] + " " + sent[idx][0]
#         if sub in movies_bn:
#             features['movie_name'] = 3
#         if sub in album_names_bn:
#             features['album_name'] = 3
#     if idx < len(sent) - 2:
#         sub = sent[idx][0] + " "+ sent[idx+1][0] + " " + sent[idx+2][0]
#         if sub in movies_bn:
#             features['movie_name'] = 3
#         if sub in album_names_bn:
#             features['album_name'] = 3
        
    
        
    
    for i in range(1, 3):
        if idx < i:
            break
        wordi = sent[idx-i][0]
        postagi = sent[idx-i][1]
        cluster_id_i = kmeans.predict([w2v.get_vector(wordi)])[0]
        features.update({
            '-{}:word'.format(i): wordi,
            '-{}:cluster_id'.format(i): cluster_id_i,
            '-{}:word[-3:]'.format(i): wordi[-3:],
            '-{}:word[-2:]'.format(i): wordi[-2:],
            '-{}:word[:3]'.format(i): wordi[:3],
            '-{}:word[:2]'.format(i): wordi[:2],
            '-{}:word.isdigit'.format(i): wordi.isdigit(),
            '-{}:postag'.format(i): postagi,
#             '-{}:is_name'.format(i) : int(wordi in names_bn)
        })
    
    for i in range(1, 3):
        if (idx+i) >= len(sent):
            break
        wordi = sent[idx+i][0]
        postagi = sent[idx+i][1]
        cluster_id_i = kmeans.predict([w2v.get_vector(wordi)])[0]
        features.update({
            '{}:word'.format(i): wordi,
            '{}:cluster_id'.format(i): cluster_id_i,
            '{}:word[-3:]'.format(i): wordi[-3:],
            '{}:word[-2:]'.format(i): wordi[-2:],
            '{}:word[:3]'.format(i): wordi[:3],
            '{}:word[:2]'.format(i): wordi[:2],
            '{}:word.isdigit'.format(i): wordi.isdigit(),
            '{}:postag'.format(i): postagi,
#             '{}:is_name'.format(i) : int(wordi in names_bn)
        })
        
    if idx == 0:
        features['BOS'] = True
    if idx == len(sent) - 1:
        features['EOS'] = True
    
    return features

def sentTofeatures(sent):
    return [wordToFeatures(sent, i) for i in range(len(sent))]

def sentTolabels(sent):
    return [label for token, postag, label in sent]

In [226]:
train_sents[0]

[('তার', 'PPR', 'O'),
 ('মৃত্যুর', 'NC', 'O'),
 ('দশ', 'JQ', 'O'),
 ('দিন', 'NC', 'O'),
 ('পর,', 'NC', 'O'),
 ('১১৫', 'RDF', 'O'),
 ('কৃষ্ণাঙ্গ', 'NC', 'O'),
 ('উচ্চ', 'JJ', 'O'),
 ('বিদ্যালয়ের', 'NC', 'O'),
 ('শিক্ষার্থীরা', 'NC', 'O'),
 ('তার', 'PPR', 'O'),
 ('হত্যার', 'NC', 'O'),
 ('প্রতিবাদে', 'NC', 'O'),
 ('ম্যাককম্ব', 'NC', 'B-LOC'),
 ('এর', 'PPR', 'O'),
 ('মাধ্যমে', 'PP', 'O'),
 ('মিছিল', 'NC', 'O'),
 ('করেছে।', 'VM', 'O')]

In [227]:
%%time
X_train = [sentTofeatures(s) for s in train_sents]
y_train = [sentTolabels(s) for s in train_sents]

CPU times: user 31min 55s, sys: 10.2 s, total: 32min 5s
Wall time: 8min 11s


In [228]:
X_train[0][3]

{'bias': 1.0,
 'word': 'দিন',
 'cluster_id': 472,
 'word[-3:]': 'দিন',
 'word[-2:]': 'িন',
 'word[:3]': 'দিন',
 'word[:2]': 'দি',
 'word.isdigit': False,
 'index': 3,
 'length': 3,
 'postag': 'NC',
 'freq': 56,
 '-1:word': 'দশ',
 '-1:cluster_id': 320,
 '-1:word[-3:]': 'দশ',
 '-1:word[-2:]': 'দশ',
 '-1:word[:3]': 'দশ',
 '-1:word[:2]': 'দশ',
 '-1:word.isdigit': False,
 '-1:postag': 'JQ',
 '-2:word': 'মৃত্যুর',
 '-2:cluster_id': 63,
 '-2:word[-3:]': 'যুর',
 '-2:word[-2:]': 'ুর',
 '-2:word[:3]': 'মৃত',
 '-2:word[:2]': 'মৃ',
 '-2:word.isdigit': False,
 '-2:postag': 'NC',
 '1:word': 'পর,',
 '1:cluster_id': 190,
 '1:word[-3:]': 'পর,',
 '1:word[-2:]': 'র,',
 '1:word[:3]': 'পর,',
 '1:word[:2]': 'পর',
 '1:word.isdigit': False,
 '1:postag': 'NC',
 '2:word': '১১৫',
 '2:cluster_id': 305,
 '2:word[-3:]': '১১৫',
 '2:word[-2:]': '১৫',
 '2:word[:3]': '১১৫',
 '2:word[:2]': '১১',
 '2:word.isdigit': True,
 '2:postag': 'RDF'}

In [229]:
# y_train

In [230]:
%%time
X_dev = [sentTofeatures(s) for s in dev_sents]
y_dev = [sentTolabels(s) for s in dev_sents]

CPU times: user 1min 43s, sys: 362 ms, total: 1min 43s
Wall time: 26.3 s


In [231]:
X_train[0][3]

{'bias': 1.0,
 'word': 'দিন',
 'cluster_id': 472,
 'word[-3:]': 'দিন',
 'word[-2:]': 'িন',
 'word[:3]': 'দিন',
 'word[:2]': 'দি',
 'word.isdigit': False,
 'index': 3,
 'length': 3,
 'postag': 'NC',
 'freq': 56,
 '-1:word': 'দশ',
 '-1:cluster_id': 320,
 '-1:word[-3:]': 'দশ',
 '-1:word[-2:]': 'দশ',
 '-1:word[:3]': 'দশ',
 '-1:word[:2]': 'দশ',
 '-1:word.isdigit': False,
 '-1:postag': 'JQ',
 '-2:word': 'মৃত্যুর',
 '-2:cluster_id': 63,
 '-2:word[-3:]': 'যুর',
 '-2:word[-2:]': 'ুর',
 '-2:word[:3]': 'মৃত',
 '-2:word[:2]': 'মৃ',
 '-2:word.isdigit': False,
 '-2:postag': 'NC',
 '1:word': 'পর,',
 '1:cluster_id': 190,
 '1:word[-3:]': 'পর,',
 '1:word[-2:]': 'র,',
 '1:word[:3]': 'পর,',
 '1:word[:2]': 'পর',
 '1:word.isdigit': False,
 '1:postag': 'NC',
 '2:word': '১১৫',
 '2:cluster_id': 305,
 '2:word[-3:]': '১১৫',
 '2:word[-2:]': '১৫',
 '2:word[:3]': '১১৫',
 '2:word[:2]': '১১',
 '2:word.isdigit': True,
 '2:postag': 'RDF'}

In [232]:
X_dev[0][0]

{'bias': 1.0,
 'word': 'তিনি',
 'cluster_id': 220,
 'word[-3:]': 'িনি',
 'word[-2:]': 'নি',
 'word[:3]': 'তিন',
 'word[:2]': 'তি',
 'word.isdigit': False,
 'index': 0,
 'length': 4,
 'postag': 'PPR',
 'freq': 2822,
 '1:word': 'যুবক',
 '1:cluster_id': 98,
 '1:word[-3:]': 'ুবক',
 '1:word[-2:]': 'বক',
 '1:word[:3]': 'যুব',
 '1:word[:2]': 'যু',
 '1:word.isdigit': False,
 '1:postag': 'NC',
 '2:word': 'হিসেবে',
 '2:cluster_id': 120,
 '2:word[-3:]': 'েবে',
 '2:word[-2:]': 'বে',
 '2:word[:3]': 'হিস',
 '2:word[:2]': 'হি',
 '2:word.isdigit': False,
 '2:postag': 'PP',
 'BOS': True}

In [233]:
import nltk
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn_crfsuite as crfsuite
from sklearn_crfsuite import metrics

In [234]:
# crf = crfsuite.CRF(
#     verbose='true',
#     algorithm='lbfgs',
#     max_iterations=100
# )

# crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)

In [ ]:
crf = crfsuite.CRF(
    verbose='true',
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=150,
    all_possible_transitions=True
)
try:
    crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
except:
    pass

loading training data to CRFsuite: 100%|██████████| 15300/15300 [00:12<00:00, 1212.31it/s]


loading dev data to CRFsuite: 100%|██████████| 800/800 [00:01<00:00, 791.06it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10

In [ ]:
# %%time
# crf = crfsuite.CRF(
#     algorithm='lbfgs',
#     c1=0.1,
#     c2=0.1,
#     max_iterations=100,
#     all_possible_transitions=True
# )
# crf.fit(X_train, y_train)

In [ ]:
labels = list(crf.classes_)
labels

In [ ]:
y_pred = crf.predict(X_dev)
metrics.flat_f1_score(y_dev, y_pred,
                      average='macro', labels=labels)

In [ ]:
def load_test(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.strip() for line in f.readlines()]
        
        sentences = []
        cur = []
        for i, line in enumerate(data):
            if line == '':
                cur = [tuple([line, ' ']) for line in cur]
                sentences.append(cur)
                cur = []
            else:
                for p in punctuations:
                    line = line.replace(p, '')
                if len(line) == 0:
                    line = ' '
                cur.append(line)
        # # convert each list to a tuple
        sentences.append(cur)
        return sentences

In [ ]:
test = load_test('../data/test.txt')
len(test)

In [ ]:
test_sents =  add_pos_to_all(test)

In [ ]:
X_test = [sentTofeatures(s) for s in test_sents]

In [ ]:
y_pred = crf.predict(X_test)
# metrics.flat_f1_score(y_dev, y_pred,
#                       average='macro', labels=labels)

In [ ]:
y_pred[0]

In [ ]:
with open("predictions_crf_with_kmeans_gazetteer.txt", "w") as file:
    for p in y_pred:
        for l in p:
            file.write(l +"\n")
        file.write("\n")
        